This is the first step in every tutorial path. We need to creat a database schema, unless it already exists.

To execute a query on Exasol database we will be using the <a href="https://github.com/exasol/pyexasol" target="_blank" rel="noopener">`pyexasol`</a> module.

TODO: Consider merging this notebook with the one where we set up the configuration. This would require changing references in other notebooks, which point to this one as a prerequisite step. 

In [3]:
# TODO: Move this to a separate configuration notebook. Here we just need to load this configuration from a store.
from dataclasses import dataclass

@dataclass
class SandboxConfig:
    EXTERNAL_HOST_NAME: str = "192.168.124.93"
    HOST_PORT: int = 8888

    @property
    def EXTERNAL_HOST(self):
        return f"""{self.EXTERNAL_HOST_NAME}:{self.HOST_PORT}"""

    USER: str = "sys"
    PASSWORD: str = "exasol"
    BUCKETFS_PORT: int = 6666
    BUCKETFS_USER: str = "w"
    BUCKETFS_PASSWORD: str = "write"
    BUCKETFS_USE_HTTPS: bool = False
    BUCKETFS_SERVICE: str = "bfsdefault"
    BUCKETFS_BUCKET: str = "default"

    @property
    def EXTERNAL_BUCKETFS_HOST(self):
        return f"""{self.EXTERNAL_HOST_NAME}:{self.BUCKETFS_PORT}"""

    @property
    def BUCKETFS_URL_PREFIX(self):
        return "https://" if self.BUCKETFS_USE_HTTPS else "http://"

    @property
    def BUCKETFS_PATH(self):
        # Filesystem-Path to the read-only mounted BucketFS inside the running UDF Container
        return self.bucketfs_path(self.BUCKETFS_BUCKET)
    
    def bucketfs_path(self, bucket: str) -> str:
        return f"/buckets/{self.BUCKETFS_SERVICE}/{bucket}"

    SCRIPT_LANGUAGE_NAME: str = "PYTHON3_60"
    UDF_FLAVOR: str = "python3-ds-EXASOL-6.0.0"
    UDF_RELEASE: str = "20190116"
    UDF_CLIENT: str = "exaudfclient" # or for newer versions of the flavor exaudfclient_py3
    SCHEMA: str = "IDA"

    @property
    def SCRIPT_LANGUAGES(self):
        return f"""{self.SCRIPT_LANGUAGE_NAME}=localzmq+protobuf:///{self.BUCKETFS_SERVICE}/
            {self.BUCKETFS_BUCKET}/{self.UDF_FLAVOR}?lang=python#buckets/{self.BUCKETFS_SERVICE}/
            {self.BUCKETFS_BUCKET}/{self.UDF_FLAVOR}/exaudf/{self.UDF_CLIENT}""";

    @property
    def connection_params(self):
        return {"dsn": self.EXTERNAL_HOST, "user": self.USER, "password": self.PASSWORD, "compression": True}

    @property
    def params(self):
        return {
            "script_languages": self.SCRIPT_LANGUAGES,
            "script_language_name": self.SCRIPT_LANGUAGE_NAME,
            "schema": self.SCHEMA,
            "BUCKETFS_PORT": self.BUCKETFS_PORT,
            "BUCKETFS_USER": self.BUCKETFS_USER,
            "BUCKETFS_PASSWORD": self.BUCKETFS_PASSWORD,
            "BUCKETFS_USE_HTTPS": self.BUCKETFS_USE_HTTPS,
            "BUCKETFS_BUCKET": self.BUCKETFS_BUCKET,
            "BUCKETFS_PATH": self.BUCKETFS_PATH
        }

#conf = SandboxConfig()

In [5]:
import pyexasol
from stopwatch import Stopwatch

def setup_schema(conf: SandboxConfig):
    stopwatch = Stopwatch()

    with pyexasol.connect(**conf.connection_params) as conn:
        conn.execute(query="CREATE SCHEMA IF NOT EXISTS {schema!i}", query_params=conf.params)

    print(f"Schema created in {stopwatch}")

# Usage 
This notebook could be used in other notebooks like this:

```
%run ../setup_db.ipynb
conf = SandboxConfig(EXTERNAL_HOST_NAME="127.0.0.1", HOST_PORT=8563)
setup_schema(conf)
```